Create a Agents using the Langchain

In [1]:
# Setup the environmental variables for the Tavily API key
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [ ]:
# Retrives TavilySearch from langchain community
# First tool of the agents that is the Tavily Search tool            
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_tool=TavilySearchResults(
    max_results=2,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    search_depth="advanced",
)

In [3]:
tavily_tool.invoke({'query': 'who won the last french open'})

[{'title': "French Open winners: Men's and women's singles champions - ESPN",
  'url': 'https://www.espn.com/tennis/story/_/id/40269330/french-open-winners-mens-womens-singles-champions',
  'content': "## ESPN\n\n# French Open winners: Men's and women's singles champions\n\nCopy Link\n\nRafael Nadal dominated the French Open since winning his first title at Roland Garros in 2005. Of the Spaniard's 22 career Grand Slam championships,14 came on the French Open's clay courts. His French Open haul eclipses any other player in tennis history.\n\nCarlos Alcaraz won the 2024 French Open, after besting Alexander Zverev in the final.",
  'score': 0.7724109,
  'raw_content': "## [ESPN](/)\n\n![](https://a3.espncdn.com/combiner/i?img=%2Fredesign%2Fassets%2Fimg%2Ficons%2FESPN%2Dicon%2Dtennis.png&w=80&h=80&scale=crop&cquality=40&location=origin)\n\n# French Open winners: Men's and women's singles champions\n\n![]()\n![Copy Link](https://a.espncdn.com/combiner/i?img=/i/content-reactions/check.png&h=

In [5]:
#Second tool for agents that is the Wikipedia
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=200)
wikipedia=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wikipedia.invoke({'query': 'who won the last french open'})

"Page: List of French Open men's singles champions\nSummary: The French Open, also known as Roland-Garros, is an annual tennis tournament held over two weeks in May and June. Established in 1891 and pla"

In [6]:
wikipedia.name

'wikipedia'

In [7]:
# Third tool for Agents LangchainSmith website
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
loader=WebBaseLoader(web_path="https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model="deepseek-r1:1.5b"))
retriever=vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


C:\Users\Anupam Maiti\AppData\Local\Temp\ipykernel_1760\1042342959.py:9: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model="deepseek-r1:1.5b"))


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DB381D4E10>, search_kwargs={})

In [8]:
# Now that we have populated our index that we will do doing retrieval over, we can easily turn it into a tool (the format needed for an agent to properly use it)
from langchain.tools.retriever import create_retriever_tool
retriver_tool=create_retriever_tool(
    retriever,
    "LangSmith_Search", 
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)

In [9]:
retriver_tool.name

'LangSmith_Search'

In [10]:
# Now that we have created both, we can create a list of tools that we will use downstream.
tools=[tavily_tool,wikipedia,retriver_tool]

In [11]:
tools

[TavilySearchResults(max_results=2, include_answer=True, include_raw_content=True, include_images=True, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Anupam Maiti\\OneDrive\\Desktop\\Python\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='LangSmith_Search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001DB506232E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DB381D4E10>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], in

In [23]:
# from  langchain_ollama import OllamaLLM
# from langchain_community.chat_models import ChatOllama
# from langchain_community.llms import Ollama
# model=Ollama(model="deepseek-r1:1.5b",temperature=1)
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI
model=ChatOpenAI(model="openai/o4-mini",base_url="https://openrouter.ai/api/v1", temperature=0.1, max_tokens=1500)

In [24]:
# We can first choose the prompt we want to use to guide the agent.

from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

from langchain_core.prompts import ChatPromptTemplate
prompts=ChatPromptTemplate.from_messages(
   [ ("system", "You are a helpful assistant. Use tools when needed. You habe to be given query based on the query you have to be search most relevent information from the tools you have been given. You can use the tools Tavily Search, Wikipedia and LangSmith Search to answer the questions asked by the user. If you are not sure about the answer, you can use the tools to search for information."),
    ("user", "{input}"),
    ("assistant", "{agent_scratchpad}"),
    ]
)

In [52]:
# If your model supports it, bind tools before creating agent
# model_with_tools = model.bind_tools(tools)

In [ ]:
# from langchain import hub
# # # Get the prompt to use - you can modify this!
# # prompt = hub.pull("hwchase17/openai-functions-agent")
# # prompt.messages
# # Create a ReAct agent instead of tool-calling agent
# from langchain.agents import create_react_agent
# prompt = hub.pull("hwchase17/react-chat")  # Special ReAct prompt
# agent = create_react_agent(model, tools, prompt)

In [25]:
print(prompt.input_variables)  # See what variables it expects

['agent_scratchpad', 'input']


In [26]:
# Now, we can initialize the agent with the LLM, the prompt, and the tools. The agent is responsible for taking in input and deciding what actions to take. Crucially, the Agent does not execute those actions - that is done by the AgentExecutor (next step).
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(model,tools,prompt)

In [27]:
# Finally, we combine the agent (the brains) with the tools inside the AgentExecutor (which will repeatedly call the agent and execute tools).
from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [30]:
# We can now run the agent on a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions).
agent_executor.invoke({"input": "hi!"} )



> Entering new AgentExecutor chain...
Hello! How can I help you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I help you today?'}

In [32]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `LangSmith_Search` with `{'query': 'LangSmith testing features'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on your data.Prompt EngineeringIterate on prompts, with automatic version control and collaboration features.
LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up L

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith provides a full suite of “testing”-focused tools to help you validate, benchmark and continuously monitor your LLM application:\n\n1. Evals (Automated & Human‐in-the-Loop)  \n   • Off-the-shelf evaluators (e.g. QA accuracy, summarization quality) you can drop in and run immediately  \n   • Custom metrics & evaluators you author against your own data  \n   • Human-feedback workflows for edge-case or qualitative checks  \n   • Batch or streaming runs against production traffic  \n\n2. Regression & Benchmarking  \n   • Save evaluation runs and compare scores over time (or across model versions)  \n   • Pinpoint regressions via historical dashboards  \n   • Automate Evals as part of your CI/CD pipeline  \n\n3. LLM-Native Observability  \n   • Trace individual prompts & outputs end-to-end (inputs, model calls, tool calls, final result)  \n   • Instrument metrics (latency, token-usage, error rates) and spin up dashboards

In [34]:
agent_executor.invoke({"input":"what is the latest news in the india?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'latest news in India June 2025'}`


[{'title': 'Current Affairs 02 June 2025 - SSBCrackExams', 'url': 'https://ssbcrackexams.com/current-affairs-02-june-2025/', 'content': "In Current Affairs for 02 June 2025, we will see the latest national and international current affairs news. These important current affairs will be. ... India's Nandini Gupta exited after making it to the final 20. Six Indian women have won the title, including Reita Faria (1966), Aishwarya Rai (1994), Diana Hayden (1997), Yukta Mookhey", 'score': 0.83742094}, {'title': 'Current Affairs 1 & 2 June 2025', 'url': 'https://affairscloud.com/current-affairs-1-2-june-2025/', 'content': 'We are here for you to provide the important Recent and Latest Current Affairs 1 & 2 June 2025, which have unique updates of Latest Current Affairs 2025 events from all newspapers such as The Hindu, The Economic Times, PIB, Times of India, PTI

{'input': 'what is the latest news in the india?',
 'output': 'Here are some of the top headlines from India as of June 3, 2025:\n\n1. Kashmir Rail Link to Open  \n  – Prime Minister Narendra Modi will inaugurate the “mighty” Chenab Bridge (the world’s highest railway bridge) on June 6 and flag off the first Vande Bharat service on the Katra–Baramulla line, finally linking Kashmir by rail to the rest of the country.\n\n2. Assam Floods Worsen  \n  – Heavy monsoon rains have displaced over 5.3 lakh people across 22 districts. The death toll stands at 11. Chief Minister Himanta Biswa Sarma toured worst-hit areas; PM Modi has assured central relief aid.\n\n3. Adani Group under U.S. Scrutiny  \n  – U.S. authorities are probing alleged links between the Adani conglomerate and sanctioned Iranian LPG trade. Adani has labelled the inquiry “baseless and mischievous.”\n\n4. India’s Coal Output Hits 86 Mt in May  \n  – Coal production rose 3.4% year-on-year to cross 86 million tonnes, reflecting s